In [ ]:
import math

class FPN:
    def __init__(self, n = 4, e = 2, m = 1, val = 0):
        self._sign = 1
        self._n = n
        self._e = e
        self._m = m
        self._Bias = (1 << e - 1) - 1
        self._Emax = (1 << (e + 1)) - 1
        self._Emin = (-1) * 2 ** (e - 1) - 1  
        self._sign = val >> (self._n - 1) 
        self._exp = (val >> self._m) & ((1 << (self._e)) - 1)
        #print(val >> m, ((1 << (self._e)) - 1))
        self._mantiss = val & ((1 << self._m) - 1)
        #print(self._sign, self._exp, self._mantiss, self._Bias)

    def __float__(self):
        if self._exp == 0 and self._mantiss == 0:
            return 0.0 * (-1) ** self._sign

        E = self._exp - self._Bias
        mant = 1.0 
        w = 0.5
        mask = 1 << (self._m - 1)
        for i in range(self._m):
            mant += w if (mask & self._mantiss == mask) else 0
            w /= 2

        if self._exp == (1 << self._e) - 1:
            if self._mantiss:
                return (-1) ** self._sign * math.inf
            else: 
                return math.nan 

        return ((-1) ** self._sign) * mant * (2 ** E)

    def __str__(self):
        return f"{self._sign:b}{self._exp:0{self._e}b}{self._mantiss:0{self._m}b}"

    def _get_mantiss(self):
        if (self._exp == 0 and self._mantiss == 0):
            return 0
        return (1 << self._m) + self._mantiss

    def __add__(self,  other):
        #можно попробовать считать в больших флотах а потом округляться к ближайшему из нашего диапазона
        res_sign = 0
        res_exp = 0
        res_mantiss = 0
        a_mantiss = 0
        b_mantiss = 0

        if  other._exp == (1 << other._e) - 1 and self._exp == (1 << self._e) - 1:
            if self._mantiss == 0:
                return FPN(self._n, self._e, self._m,
                        self._sign << (self._n - 1) | self._exp << self._m | self._mantiss)
            else:
                return FPN(other._n, other._e, other._m, 
                        (other._sign << (other._n - 1)) | other._exp << other._m | other._mantiss)

        if other._exp == (1 << other._e) - 1:
            return FPN(other._n, other._e, other._m, 
                        (other._sign << (other._n - 1)) | other._exp << other._m | other._mantiss)
            
        if self._exp == (1 << self._e) - 1:
            return FPN(self._n, self._e, self._m,
                        self._sign << (self._n - 1) | self._exp << self._m | self._mantiss)


        if self._exp > other._exp:
            dif_e = self._exp - other._exp
            res_exp = self._exp
            b_mantiss = (other._get_mantiss()) >> dif_e
            a_mantiss = (self._get_mantiss())
        else:
            dif_e = other._exp - self._exp
            res_exp = other._exp
            a_mantiss = (self._get_mantiss()) >> dif_e
            b_mantiss = other._get_mantiss()
        
        val_a = ((a_mantiss) * (-1) ** self._sign)
        val_b = (b_mantiss) * (-1) ** other._sign 
        res = val_a + val_b
       

        print(bin(res), bin(val_a), bin(val_b), bin(a_mantiss), bin(b_mantiss))

        if res == 0:
            res_sign = 1 if self._sign == 1 else 0
            return FPN(self._n, self._e, self._m, res_sign << self._n - 1)
        
        #if (self._sign != other._sign):
            #res = res & ((1 << (self._m + 1) ) - 1)
        
        res_sign = 0 if res > 0 else 1
        res_mantiss = abs(res)

        while ((1 << (self._m + 1) - 1) < res_mantiss):
            res_mantiss >>= 1
            res_exp += 1

        while (1 << self._m > res_mantiss):
            res_mantiss <<= 1
            res_exp -= 1

        if res_exp == (1 << self._e) - 1:
            res_mantiss = (1 << self._m) - 1

        return FPN(self._n, self._e, self._m,
                    val= (res_sign << self._n - 1) | (res_exp << self._m) | ((res_mantiss & (1 << self._m) - 1)))

    def __neg__(self):
        return FPN(n=self._n, e=self._e, m=self._m, 
                   val=(self._sign ^ 1) << self._n - 1 | (self._exp << self._m) | ((self._mantiss & (1 << self._m) - 1)))

    def __sub__(self, other):
        return self + (-other)



In [93]:
for a in range(1 << 4):
    fp4_X = FPN(val=a)
    print(a, fp4_X, float(fp4_X))
    print("-------")

0 0000 0.0
-------
1 0001 0.75
-------
2 0010 1.0
-------
3 0011 1.5
-------
4 0100 2.0
-------
5 0101 3.0
-------
6 0110 nan
-------
7 0111 inf
-------
8 1000 -0.0
-------
9 1001 -0.75
-------
10 1010 -1.0
-------
11 1011 -1.5
-------
12 1100 -2.0
-------
13 1101 -3.0
-------
14 1110 nan
-------
15 1111 -inf
-------


In [97]:
for a in range(1 << 4):
    fp4_X = FPN(val=a)
    nfp4_X = -fp4_X
    print(a, fp4_X, float(fp4_X))
    print(a, (-fp4_X), float(fp4_X))
    print("-------")

TypeError: unsupported operand type(s) for |: 'FPN' and 'int'

In [ ]:
for a in range(1 << 4):
    for b in range(1 << 4):
        fp4_X_a = FPN(val=a)
        print(a, fp4_X_a, float(fp4_X_a))
        fp4_X_b = FPN(val=b)
        print(b, fp4_X_b, float(fp4_X_b))
        fp4_X_sum = fp4_X_a + fp4_X_b
        print("sum:", fp4_X_sum, float(fp4_X_sum))
        print("-------")

0 0000 0.0
0 0000 0.0
0b0 0b0 0b0 0b0 0b0
sum: 0000 0.0
-------
0 0000 0.0
1 0001 0.75
0b11 0b0 0b11 0b0 0b11
sum: 0000 0.0
-------
0 0000 0.0
2 0010 1.0
0b10 0b0 0b10 0b0 0b10
sum: 0010 1.0
-------
0 0000 0.0
3 0011 1.5
0b11 0b0 0b11 0b0 0b11
sum: 0010 1.0
-------
0 0000 0.0
4 0100 2.0
0b10 0b0 0b10 0b0 0b10
sum: 0100 2.0
-------
0 0000 0.0
5 0101 3.0
0b11 0b0 0b11 0b0 0b11
sum: 0100 2.0
-------
0 0000 0.0
6 0110 nan
sum: 0110 nan
-------
0 0000 0.0
7 0111 inf
sum: 0111 inf
-------
0 0000 0.0
8 1000 -0.0
0b0 0b0 0b0 0b0 0b0
sum: 0000 0.0
-------
0 0000 0.0
9 1001 -0.75
-0b11 0b0 -0b11 0b0 0b11
sum: 1000 -0.0
-------
0 0000 0.0
10 1010 -1.0
-0b10 0b0 -0b10 0b0 0b10
sum: 1010 -1.0
-------
0 0000 0.0
11 1011 -1.5
-0b11 0b0 -0b11 0b0 0b11
sum: 1010 -1.0
-------
0 0000 0.0
12 1100 -2.0
-0b10 0b0 -0b10 0b0 0b10
sum: 1100 -2.0
-------
0 0000 0.0
13 1101 -3.0
-0b11 0b0 -0b11 0b0 0b11
sum: 1100 -2.0
-------
0 0000 0.0
14 1110 nan
sum: 1110 nan
-------
0 0000 0.0
15 1111 -inf
sum: 1111 -inf
---

In [ ]:
a = 12
b = 3
fp4_X_a = FPN(val=a)
print(a, fp4_X_a, float(fp4_X_a))
fp4_X_b = FPN(val=b)
print(b, fp4_X_b, float(fp4_X_b))
fp4_X_sum = fp4_X_a + fp4_X_b
print("sum:", fp4_X_sum, float(fp4_X_sum))
print("-------")

12 1100 -2.0
3 0011 1.5
-0b1 -0b10 0b1 0b10 0b1
sum: 1010 -1.0
-------


In [ ]:
a = 9
b = 0
fp4_X_a = FPN(val=a)
print(a, fp4_X_a, float(fp4_X_a))
fp4_X_b = FPN(val=b)
print(b, fp4_X_b, float(fp4_X_b))
fp4_X_sum = fp4_X_a + fp4_X_b
print("sum:", fp4_X_sum, float(fp4_X_sum))
print("-------")

9 1001 -0.75
0 0000 0.0
-0b11 -0b11 0b0 0b11 0b0
sum: 1000 -0.0
-------


In [ ]:
a = 12
b = 8
fp4_X_a = FPN(val=a)
print(a, fp4_X_a, float(fp4_X_a))
fp4_X_b = FPN(val=b)
print(b, fp4_X_b, float(fp4_X_b))
fp4_X_sum = fp4_X_a + fp4_X_b
print("sum:", fp4_X_sum, float(fp4_X_sum))
print("-------")

12 1100 -2.0
8 1000 -0.0
-0b10 -0b10 0b0 0b10 0b0
sum: 1100 -2.0
-------


In [ ]:
a = 13
b = 9
fp4_X_a = FPN(val=a)
print(a, fp4_X_a, float(fp4_X_a))
fp4_X_b = FPN(val=b)
print(b, fp4_X_b, float(fp4_X_b))
fp4_X_sum = fp4_X_a + fp4_X_b
print("sum:", fp4_X_sum, float(fp4_X_sum))
print("-------")

13 1101 -3.0
9 1001 -0.75
-0b11 -0b11 0b0 0b11 0b0
sum: 1100 -2.0
-------


In [ ]:
a = 12
b = 13
fp4_X_a = FPN(val=a)
print(a, fp4_X_a, float(fp4_X_a))
fp4_X_b = FPN(val=b)
print(b, fp4_X_b, float(fp4_X_b))
fp4_X_sum = fp4_X_a + fp4_X_b
print("sum:", fp4_X_sum, float(fp4_X_sum))
print("-------")

12 1100 -2.0
13 1101 -3.0
-0b101 -0b10 -0b11 0b10 0b11
sum: 1111 -inf
-------


In [ ]:
for a in range(1 << 4):
    for b in range(1 << 4):
        fp4_X_a = FPN(val=a)
        print(a, fp4_X_a, float(fp4_X_a))
        fp4_X_b = FPN(val=b)
        print(b, fp4_X_b, float(fp4_X_b))
        fp4_X_sum = fp4_X_a - fp4_X_b
        print("sub:", fp4_X_sum, float(fp4_X_sum))
        print("-------")

0 0000 0.0
0 0000 0.0
0b0 0b0 0b0 0b0 0b0
sub: 0000 0.0
-------
0 0000 0.0
1 0001 0.75
-0b11 0b0 -0b11 0b0 0b11
sub: 1000 -0.0
-------
0 0000 0.0
2 0010 1.0
-0b10 0b0 -0b10 0b0 0b10
sub: 1010 -1.0
-------
0 0000 0.0
3 0011 1.5
-0b11 0b0 -0b11 0b0 0b11
sub: 1010 -1.0
-------
0 0000 0.0
4 0100 2.0
-0b10 0b0 -0b10 0b0 0b10
sub: 1100 -2.0
-------
0 0000 0.0
5 0101 3.0
-0b11 0b0 -0b11 0b0 0b11
sub: 1100 -2.0
-------
0 0000 0.0
6 0110 nan
sub: 1110 nan
-------
0 0000 0.0
7 0111 inf
sub: 1111 -inf
-------
0 0000 0.0
8 1000 -0.0
0b0 0b0 0b0 0b0 0b0
sub: 0000 0.0
-------
0 0000 0.0
9 1001 -0.75
-0b11 0b0 -0b11 0b0 0b11
sub: 1000 -0.0
-------
0 0000 0.0
10 1010 -1.0
-0b10 0b0 -0b10 0b0 0b10
sub: 1010 -1.0
-------
0 0000 0.0
11 1011 -1.5
-0b11 0b0 -0b11 0b0 0b11
sub: 1010 -1.0
-------
0 0000 0.0
12 1100 -2.0
-0b10 0b0 -0b10 0b0 0b10
sub: 1100 -2.0
-------
0 0000 0.0
13 1101 -3.0
-0b11 0b0 -0b11 0b0 0b11
sub: 1100 -2.0
-------
0 0000 0.0
14 1110 nan
sub: 1110 nan
-------
0 0000 0.0
15 1111 -inf
su

In [ ]:
a = 13
b = 13
fp4_X_a = FPN(val=a)
print(a, fp4_X_a, float(fp4_X_a))
fp4_X_b = FPN(val=b)
print(b, fp4_X_b, float(fp4_X_b))
fp4_X_sub = fp4_X_a - fp4_X_b
print("sub:", fp4_X_sub, float(fp4_X_sub))
print("-------")

13 1101 -3.0
13 1101 -3.0


KeyboardInterrupt: 